In [4]:
import pandas as pd
import geopandas as gpd

from shapely import wkt


In [3]:
df = pd.read_csv('../data/clean/h3_PLUTO.csv', low_memory=False)


In [4]:
gdf = gpd.GeoDataFrame(df)

In [5]:
poi_df = pd.read_csv('../data/raw/points_of_interest.csv')
bbl_to_h3 = pd.read_json('../data/clean//bbl_to_h3.json', orient='index')



In [6]:
poi_df = poi_df.rename({'the_geom': 'geometry'}, axis=1)
poi_df['geometry'] = poi_df['geometry'].apply(wkt.loads)
poi_gdf = gpd.GeoDataFrame(poi_df, crs='epsg:4326')

TypeError: Only str is accepted.

In [8]:
poi_h3 = poi_gdf.h3.geo_to_h3(10)

poi_h3.to_csv('../data/clean/poi_h3.csv')

NameError: name 'poi_gdf' is not defined

In [5]:
poi_h3 = gpd.read_file('../data/clean/poi_h3.csv')

In [6]:
poi_hex_df = pd.DataFrame()
poi_hex_df['NAME'] = poi_h3.groupby('h3_10').agg(list)['NAME']
# poi_hexes.rename(['h3_10','NAME'],axis=1)
# poi_hexes['COUNT'] = len(poi_hexes['NAME'])



In [7]:
poi_hex_df = pd.DataFrame(poi_hex_df[:-1])
poi_hex_df['COUNT'] = poi_hex_df['NAME'].apply(len)
# 
poi_hex_df.sort_values(['COUNT'])


,NAME,COUNT
h3_10,,
8a2a1000005ffff,[RAT ISLAND],1
8a2a100f3a87fff,[BRIGHT BEGINNINGS 30 AV],1
8a2a100f3b47fff,[I-278 BQE EAST LEG OVER 30 AVE],1
8a2a100f3b5ffff,[ST MICHAELS PLAYGROUND SOCCER FIELD],1
8a2a100f3baffff,[BULOVA AVENUE OV I-278 BQE W LEG],1
...,...,...
8a2a100d622ffff,"[ESTONIA CONSULATE, MONTESSORI FAMILY SCHOOL, ...",16
8a2a100d62e7fff,"[NICARAGUA CONSULAT, SYRIA CONSULATE, NEPAL MI...",20
8a2a100d9167fff,"[HOWARD-PARK HOUSES, HOWARD AVENUE PARK PLACE ...",22


In [8]:
# visualize_hexagons(list(poi_h3.reset_index()['h3_10']))

In [9]:

poi_hex_df['hex_id'] = poi_hex_df.index
df = pd.read_csv('../data/clean/h3_PLUTO.csv', low_memory=False)
# pls = gpd.GeoDataFrame(poi_hex_df.h3.h3_to_geo_boundary(),Proj("epsg:4326"))

In [19]:
# from hex_config import config
from keplergl import KeplerGl
% run hex_config.py

df_gb = df.groupby('h3_10').agg({'BldgArea': 'sum', 'YearBuilt': 'mean'})
df_gb['hex_id'] = df_gb.index

df_gb
# df_gb = df_gb.set_index(['hex_id'])

,BldgArea,YearBuilt,hex_id
h3_10,,,
8a2a1000009ffff,0,0.00000,8a2a1000009ffff
8a2a10000287fff,4456,1947.50000,8a2a10000287fff
8a2a1000028ffff,596,1932.50000,8a2a1000028ffff
8a2a10000297fff,41770,1862.73913,8a2a10000297fff
8a2a1000029ffff,20394,1808.00000,8a2a1000029ffff
...,...,...,...
8a2a10776d87fff,111265,1920.00000,8a2a10776d87fff
8a2a10776d8ffff,227000,950.00000,8a2a10776d8ffff
8a2a10776da7fff,79000,1958.50000,8a2a10776da7fff


In [20]:

# map_1.save_to_html(file_name='../out/html/kepler_test.html')
map_1 = KeplerGl(height=400, data={'data_poi': poi_hex_df, 'df_gb': df_gb}, config=config)

map_1.save_to_html(file_name="helpme.html")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to helpme.html!


In [9]:
import folium
from h3 import h3


def visualize_hexagons(hexagons, color="red", folium_map=None, clipper=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # polygons1=polygons.copy()
        # for i in polygons1:
        # gpd.GeoDataFrame(polygons1).clip(clipper)

        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v: v[0], polyline))
        lng.extend(map(lambda v: v[1], polyline))
        polylines.append(polyline)

    if folium_map is None:
        # m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron',prefer_canvas=True)
        m = folium.Map(location=[sum(lat) / len(lat), sum(lng) / len(lng)], zoom_start=11)
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine = folium.PolyLine(locations=polyline, weight=2, color=color, )
        m.add_child(my_PolyLine)

    return m


def visualize_polygon(polyline, color="red", folium_map=None):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    if folium_map is None:
        m = folium.Map(location=[sum(lat) / len(lat), sum(lng) / len(lng)], tiles='cartodbpositron')
    else:
        m = folium_map
    my_PolyLine = folium.PolyLine(locations=polyline, weight=2, color=color)
    m.add_child(my_PolyLine)
    return m

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'cuia8dl',
     'type': 'geojson',
     'config': {'dataId': 'df_w_wkt',
      'label': 'df_w_wkt',
      'color': [30, 150, 190],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'wkt_string'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [137, 218, 193],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusR